In [1]:
import pandas as pd
import numpy as np
import ast
import json

Se cargan las bases de datos que se construyeron en el cuaderno etl_eda.ipynb

In [2]:
reviews=pd.read_parquet(r'reviews.parquet')
games=pd.read_parquet(r'games.parquet')
users=pd.read_parquet(r'users.parquet')

Creacion del dataset developers

In [3]:
games.head(2)

,genres,app_name,tags,price,id,developer,release_anio
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,643980,Secret Level Srl,2018


In [4]:
reviews.head(2)

,user_id,funny,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,0,1250,0,1,2
1,js41637,0,251610,15,1,2


Se crea dataset developers

In [5]:
#Conteo de la columna sentiment analisis por item
sentiment=reviews.groupby('item_id')['sentiment_analysis'].value_counts().unstack().reset_index()

In [6]:
sentiment.head()

sentiment_analysis,item_id,0,1,2
0,10,NaN,12.0,12.0
1,20,NaN,2.0,7.0
2,40,NaN,NaN,1.0
3,50,NaN,NaN,1.0
4,60,1.0,NaN,3.0


In [7]:
# conteo de recomendaciones por item
recomment=reviews.groupby('item_id')['recommend'].value_counts().unstack().reset_index()

In [8]:
recomment.head()

recommend,item_id,0,1
0,10,NaN,24.0
1,20,3.0,6.0
2,40,NaN,1.0
3,50,NaN,1.0
4,60,NaN,4.0


In [9]:
recomment.rename(columns={0: 'No', 1:'Yep'}, inplace=True)

In [10]:
recomment.head()

recommend,item_id,No,Yep
0,10,NaN,24.0
1,20,3.0,6.0
2,40,NaN,1.0
3,50,NaN,1.0
4,60,NaN,4.0


Ahora se cruzan los dos dataframe usando el item_id

In [11]:
impressions=pd.merge(sentiment, recomment, on='item_id')

In [12]:
impressions.head()

,item_id,0,1,2,No,Yep
0,10,NaN,12.0,12.0,NaN,24.0
1,20,NaN,2.0,7.0,3.0,6.0
2,40,NaN,NaN,1.0,NaN,1.0
3,50,NaN,NaN,1.0,NaN,1.0
4,60,1.0,NaN,3.0,NaN,4.0


In [13]:
impressions.fillna(0,inplace=True)

In [14]:
impressions=impressions.astype(int)

In [15]:
impressions.head()

,item_id,0,1,2,No,Yep
0,10,0,12,12,0,24
1,20,0,2,7,3,6
2,40,0,0,1,0,1
3,50,0,0,1,0,1
4,60,1,0,3,0,4


Se unen los dataframe games e impressions

In [16]:
developers=pd.merge(games[['price','developer','release_anio','id']], impressions, left_on='id',right_on='item_id')

In [17]:
developers.head()

,price,developer,release_anio,id,item_id,0,1,2,No,Yep
0,9.99,Stainless Games Ltd,1997,282010,282010,0,1,0,0,1
1,9.99,Valve,1998,70,70,1,2,16,0,19
2,9.99,Outerlight Ltd.,2006,2400,2400,2,1,7,0,10
3,9.99,Introversion Software,2006,1520,1520,0,0,3,0,3
4,9.99,Outerlight Ltd.,2006,2420,2420,0,0,1,0,1


In [18]:
developers.drop('id',axis=1,inplace=True)

In [19]:
developers.head()

,price,developer,release_anio,item_id,0,1,2,No,Yep
0,9.99,Stainless Games Ltd,1997,282010,0,1,0,0,1
1,9.99,Valve,1998,70,1,2,16,0,19
2,9.99,Outerlight Ltd.,2006,2400,2,1,7,0,10
3,9.99,Introversion Software,2006,1520,0,0,3,0,3
4,9.99,Outerlight Ltd.,2006,2420,0,0,1,0,1


Se renombran las columnas

In [20]:
developers.rename(columns={0:'Negative',1:'Neutral',2:'Positive','No':'False','Yep':'True'},inplace=True)

In [22]:
developers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013 entries, 0 to 2012
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         2013 non-null   float64
 1   developer     2013 non-null   object 
 2   release_anio  2013 non-null   object 
 3   item_id       2013 non-null   int32  
 4   Negative      2013 non-null   int32  
 5   Neutral       2013 non-null   int32  
 6   Positive      2013 non-null   int32  
 7   False         2013 non-null   int32  
 8   True          2013 non-null   int32  
dtypes: float64(1), int32(6), object(2)
memory usage: 94.5+ KB


In [23]:
developers['release_anio'].unique()

array(['1997', '1998', '2006', '2005', '1996', '1995', '1994', '2007',
       '2004', '2003', '2001', '2008', '1993', '2009', '2002', '2010',
       '2011', 'Feb 2011', '2013', '2012', '2015', '2014', '1992', '2016',
       '1989', '1999', '2017', 'Oct 2016', 'Jun 2015', 'Nov 2014',
       'Jul 2014', 'Mar 2010', '2000', 'Sep 2009', '1990'], dtype=object)

In [26]:
developers['release_anio'] = developers['release_anio'].str.replace(r'[A-Za-z]', '', regex=True).astype(int)

In [27]:
developers['release_anio'].unique()

array([1997, 1998, 2006, 2005, 1996, 1995, 1994, 2007, 2004, 2003, 2001,
       2008, 1993, 2009, 2002, 2010, 2011, 2013, 2012, 2015, 2014, 1992,
       2016, 1989, 1999, 2017, 2000, 1990])

In [28]:
developers.describe()

,price,release_anio,item_id,Negative,Neutral,Positive,False,True
count,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000
mean,12.471401,2012.662692,247565.246398,1.534526,2.213612,6.883756,1.051664,9.580229
std,12.755088,3.913831,131208.638719,8.436727,19.602209,49.085690,5.178073,72.777604
min,0.000000,1989.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.990000,2012.000000,207250.000000,0.000000,0.000000,1.000000,0.000000,1.000000
50%,9.990000,2014.000000,265120.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,19.980000,2015.000000,340170.000000,1.000000,1.000000,3.000000,1.000000,4.000000
max,190.000000,2017.000000,527340.000000,286.000000,619.000000,1489.000000,131.000000,2263.000000


Se guarda el dataset en formato parquet

In [29]:
developers.to_parquet(r'developers.parquet')

Dataset users

In [30]:
users.head(2)

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0


In [31]:
users['user_id']=users['user_id'].astype('str')

Se crea el df1 de la unión de los juegos y los usuarios y se descartan las columnas que no se necesitan en las consultas.


In [41]:
df1=pd.merge(games,users,left_on='id',right_on='item_id',how= 'inner')
df1.drop(columns=['id','developer','app_name','tags'],inplace=True)
df1.drop(columns=['item_id'],inplace=True)

In [42]:
df1.head()

,genres,price,release_anio,user_id,items_count,playtime_forever
0,"['Action', 'Indie', 'Racing']",9.99,1997,UTNerd24,188,5.0
1,"['Action', 'Indie', 'Racing']",9.99,1997,I_DID_911_JUST_SAYING,154,0.0
2,"['Action', 'Indie', 'Racing']",9.99,1997,76561197962104795,79,0.0
3,"['Action', 'Indie', 'Racing']",9.99,1997,r3ap3r78,331,0.0
4,"['Action', 'Indie', 'Racing']",9.99,1997,saint556,583,13.0


Se toma solo el primer genero como principal para la consulta.

Se define funcion solo para extraer la primera palabra que este dentro de unas comillas simples de toda la cadena de texto

In [43]:
import re

In [45]:
def extraer_comillas_singles(texto):
    match = re.search(r"'(.*?)'", texto)
    return match.group(1) if match else None

In [47]:
df1['genres']=df1['genres'].apply(extraer_comillas_singles)

In [48]:
df1.head()

,genres,price,release_anio,user_id,items_count,playtime_forever
0,Action,9.99,1997,UTNerd24,188,5.0
1,Action,9.99,1997,I_DID_911_JUST_SAYING,154,0.0
2,Action,9.99,1997,76561197962104795,79,0.0
3,Action,9.99,1997,r3ap3r78,331,0.0
4,Action,9.99,1997,saint556,583,13.0


Se crea un nuevo dataframe con la cantidad de items por usuario

In [49]:
df_grouped = df1.groupby(['release_anio', 'items_count', 'user_id','genres']).sum().reset_index()

In [50]:
df_grouped.head()

,release_anio,items_count,user_id,genres,price,playtime_forever
0,1983,2,76561198096490892,Action,9.99,2.0
1,1983,31,TonkaTuff,Action,9.99,53.0
2,1983,38,PotatoesBasket,Action,9.99,0.0
3,1983,43,76561198064710856,Action,9.99,37.0
4,1983,51,76561197985271912,Action,9.99,3.0


In [51]:
df_grouped[df_grouped['user_id']=='TonkaTuff']

,release_anio,items_count,user_id,genres,price,playtime_forever
1,1983,31,TonkaTuff,Action,9.99,53.0
35913,1998,31,TonkaTuff,Action,9.99,0.0
53476,1999,31,TonkaTuff,Action,9.98,0.0
69382,2000,31,TonkaTuff,Action,14.98,0.0
84735,2001,31,TonkaTuff,Action,9.98,0.0
121700,2003,31,TonkaTuff,Action,4.99,0.0
157685,2004,31,TonkaTuff,Action,34.96,9.0
241967,2006,31,TonkaTuff,Action,17.98,0.0
323716,2007,31,TonkaTuff,Action,17.98,0.0
444830,2009,31,TonkaTuff,Action,9.99,66.0


Se crea el parquet

In [52]:
df_grouped.to_parquet(r'users_grouped.parquet')

Dataset users_recommends

In [115]:
df2=reviews.copy()

In [116]:
df2.head()

,user_id,funny,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,0,1250,0,1,2
1,js41637,0,251610,15,1,2
2,evcentric,0,248820,0,1,2
3,doctr,0,250320,2,1,2
4,maplemage,3,211420,35,1,1


In [117]:
df2=df2.drop(columns=(['item_id']),axis=1)

In [119]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25771 entries, 0 to 25770
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25771 non-null  object
 1   funny               25771 non-null  int64 
 2   helpful             25771 non-null  int64 
 3   recommend           25771 non-null  int64 
 4   sentiment_analysis  25771 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1006.8+ KB


In [120]:
total=df2.groupby('user_id')['recommend'].count()

In [121]:
total.head()

user_id
--000--         1
--ace--         1
--ionex--       1
-2SV-vuLB-Kg    1
-Azsael-        1
Name: recommend, dtype: int64

In [125]:
total.unique()

array([1, 2, 3], dtype=int64)

In [126]:
positivos=df2[df2['recommend']==1].groupby('user_id')['recommend'].count()
positivos=positivos.reindex(total.index,fill_value=0)

In [128]:
porcentaje_positivos=positivos*100/total

In [129]:
porcentaje_positivos

user_id
--000--         100.0
--ace--         100.0
--ionex--       100.0
-2SV-vuLB-Kg    100.0
-Azsael-        100.0
                ...  
zwanzigdrei     100.0
zy0705          100.0
zynxgameth      100.0
zyr0n1c         100.0
zzoptimuszz     100.0
Name: recommend, Length: 25458, dtype: float64

In [130]:
df2=pd.merge(df2,porcentaje_positivos,on='user_id')

In [131]:
df2.head()

,user_id,funny,helpful,recommend_x,sentiment_analysis,recommend_y
0,76561197970982479,0,0,1,2,100.0
1,js41637,0,15,1,2,100.0
2,evcentric,0,0,1,2,100.0
3,doctr,0,2,1,2,100.0
4,maplemage,3,35,1,1,100.0


In [132]:
df2.drop_duplicates(inplace=True)

In [133]:
df2.head()

,user_id,funny,helpful,recommend_x,sentiment_analysis,recommend_y
0,76561197970982479,0,0,1,2,100.0
1,js41637,0,15,1,2,100.0
2,evcentric,0,0,1,2,100.0
3,doctr,0,2,1,2,100.0
4,maplemage,3,35,1,1,100.0


In [134]:
df2.drop(columns=['recommend_x'],inplace=True)

In [135]:
df2.rename(columns={'recommend_y':'perc_recomm'},inplace=True)

In [136]:
df2['perc_recomm']=df2['perc_recomm'].astype('str')+'%'

In [137]:
df2['user_id']=df2['user_id'].astype('str')

In [138]:
df2.head()

,user_id,funny,helpful,sentiment_analysis,perc_recomm
0,76561197970982479,0,0,2,100.0%
1,js41637,0,15,2,100.0%
2,evcentric,0,0,2,100.0%
3,doctr,0,2,2,100.0%
4,maplemage,3,35,1,100.0%


In [89]:
df2['perc_recomm'].unique()

array(['100.0%', '0.0%'], dtype=object)

In [139]:
df2.to_parquet(r'user_recommends.parquet')

Dataset ML

In [148]:
df=games.copy()

In [149]:
df.drop(columns=['developer','price'],inplace=True)

Se toma solo el primer genero

In [151]:
df['genres']=df['genres'].apply(extraer_comillas_singles)

Se toma solo el primer tag

In [154]:
df['tags']=df['tags'].apply(extraer_comillas_singles)

In [158]:
df['release_anio'].unique()

array(['2018', '2017', '1997', '1998', '2016', '2006', '2005', '2003',
       '2007', '2002', '2000', '1995', '1996', '1994', '2001', '1993',
       '2004', '2008', '2009', 'Jun 2009', '1999', '1992', '1989', '2010',
       '2011', 'Oct 2010', 'Feb 2011', '2013', '2012', '2014', '1983',
       '1984', '2015', 'Sep 2014', '1990', '1988', '1991', 'Apr 2015',
       'Apr 2016', 'Jul 2016', '1987', 'Jul 2017', '1986', '2021', '2019',
       'Apr 2017', 'Jan 2017', 'Nov 2016', 'Oct 2016', 'Jun 2016',
       'Aug 2015', 'Jun 2015', 'May 2015', 'Feb 2015', 'Jan 2015', '1985',
       'Nov 2014', 'Aug 2014', 'Jul 2014', 'May 2014', 'Feb 2013',
       'Dec 2012', 'Jul 2010', 'Mar 2010', 'Jan 2010', 'Oct 2009',
       'Sep 2009'], dtype=object)

Se limpia la columna release_anio

In [159]:
df['release_anio'] = df['release_anio'].str.replace(r'[A-Za-z]', '', regex=True).astype(int)

In [160]:
df['release_anio'].unique()

array([2018, 2017, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000,
       1995, 1996, 1994, 2001, 1993, 2004, 2008, 2009, 1999, 1992, 1989,
       2010, 2011, 2013, 2012, 2014, 1983, 1984, 2015, 1990, 1988, 1991,
       1987, 1986, 2021, 2019, 1985])

In [161]:
df.to_parquet(r'data_ML.parquet')